# MLP inference with TT-NN

In this example we will combine insight from the previous examples, and
use TT-NN with PyTorch to perform a simple MLP inference task. This will
demonstrate how to use TT-NN for tensor operations and model inference.

Lets create the example file, `ttnn_mlp_inference_mnist.py`

## Import the necessary libraries

Amongst these, torchvision is used to load the MNIST dataset, and ttnn
is used for tensor operations on the Tenstorrent device.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import ttnn
import os
from loguru import logger

## Open Tenstorrent device

Create necessary device on which we will run our program.

In [ ]:
# Open Tenstorrent device
device = ttnn.open_device(device_id=0)

## Load MNIST Test Data

Load and convert the MNIST 28x28 grayscale images to tensors and
normalize them. Subsequently, lets create a DataLoader to iterate
through the dataset. This will allow us to perform inference on each
image in the dataset.

In [ ]:
# Load MNIST data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
testset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

## Load Pretrained MLP Weights

Load the pretrained MLP weights from a file.

In [ ]:
if os.path.exists("mlp_mnist_weights.pt"):
    # Pretrained weights
    weights = torch.load("mlp_mnist_weights.pt")
    W1 = weights["W1"]
    b1 = weights["b1"]
    W2 = weights["W2"]
    b2 = weights["b2"]
    W3 = weights["W3"]
    b3 = weights["b3"]
    logger.info("Loaded pretrained weights from mlp_mnist_weights.pt")
else:
    # Random weights for MLP - will not predict correctly
    logger.warning("mlp_mnist_weights.pt not found, using random weights")
    torch.manual_seed(0)
    W1 = torch.randn((128, 28 * 28), dtype=torch.float32)
    b1 = torch.randn((128,), dtype=torch.float32)
    W2 = torch.randn((64, 128), dtype=torch.float32)
    b2 = torch.randn((64,), dtype=torch.float32)
    W3 = torch.randn((10, 64), dtype=torch.float32)
    b3 = torch.randn((10,), dtype=torch.float32)

## Basic accuracy tracking, inference, loop, and image flattening

Loop through the first 5 images in the data set, and convert the image
from 1x28x28 to 1x784 by flattening it. This is done to match the input
shape of the MLP model.

In [ ]:
correct = 0
total = 0

for i, (image, label) in enumerate(testloader):
    if i >= 5:
        break

    image = image.view(1, -1).to(torch.float32)
    
    # Convert to TT-NN Tensor
    # Convert the PyTorch tensor to TT-NN format with bfloat16 data type and
    # TILE\_LAYOUT. This is necessary for efficient computation on the
    # Tenstorrent device.
    image_tt = ttnn.from_torch(image, dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    image_tt = ttnn.to_layout(image_tt, ttnn.TILE_LAYOUT)
    
    # Layer 1
    # Transposed weights are used to match TT-NN's expected shape. Bias
    # reshaped to 1x128 for broadcasting, and compute output 1.
    W1_tt = ttnn.from_torch(W1.T, dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    W1_tt = ttnn.to_layout(W1_tt, ttnn.TILE_LAYOUT)
    b1_tt = ttnn.from_torch(b1.view(1, -1), dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    b1_tt = ttnn.to_layout(b1_tt, ttnn.TILE_LAYOUT)
    out1 = ttnn.linear(image_tt, W1_tt, bias=b1_tt)
    out1 = ttnn.relu(out1)
    
    # Layer 2
    # Same pattern as Layer 1, but with different weights and biases.
    W2_tt = ttnn.from_torch(W2.T, dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    W2_tt = ttnn.to_layout(W2_tt, ttnn.TILE_LAYOUT)
    b2_tt = ttnn.from_torch(b2.view(1, -1), dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    b2_tt = ttnn.to_layout(b2_tt, ttnn.TILE_LAYOUT)
    out2 = ttnn.linear(out1, W2_tt, bias=b2_tt)
    out2 = ttnn.relu(out2)
    
    # Layer 3
    # Final layer with 10 output (for digits 0-9). No ReLU activation here, as
    # this is the output layer.
    W3_tt = ttnn.from_torch(W3.T, dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    W3_tt = ttnn.to_layout(W3_tt, ttnn.TILE_LAYOUT)
    b3_tt = ttnn.from_torch(b3.view(1, -1), dtype=ttnn.bfloat16, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    b3_tt = ttnn.to_layout(b3_tt, ttnn.TILE_LAYOUT)
    out3 = ttnn.linear(out2, W3_tt, bias=b3_tt)
    
    # Convert result back to torch
    prediction = ttnn.to_torch(out3)
    predicted_label = torch.argmax(prediction, dim=1).item()
    
    correct += predicted_label == label.item()
    total += 1
    
    logger.info(f"Sample {i+1}: Predicted={predicted_label}, Actual={label.item()}")
    
logger.info(f"\nTT-NN MLP Inference Accuracy: {correct}/{total} = {100.0 * correct / total:.2f}%")

## Full example and output

Lets put everything together in a complete example that can be run
directly. This example will open a Tenstorrent device, create two
tensors, perform the addition, and log the output tensor. You can run
the provided `train_and_export_mlp.py` script to generate the weights to
a file named `mlp_mnist_weights.pt`.

## Close the device

In [ ]:
ttnn.close_device(device)